# Data preparation

**SageMaker Studio Kernel**: Data Science

The challenge we're trying to address here is to detect anomalies in the components of a Wind Turbine. Each wind turbine has many sensors that reads data like:
 - Internal & external temperature
 - Wind speed
 - Rotor speed
 - Air pressure
 - Voltage (or current) in the generator
 - Vibration in the GearBox (using an IMU -> Accelerometer + Gyroscope)

So, depending on the types of the anomalies we want to detect, we need to select one or more features and then prepare a dataset that 'explains' the anomalies. We are interested in three types of anomalies:
 - Rotor speed (when the rotor is not in an expected speed)
 - Produced voltage (when the generator is not producing the expected voltage)
 - Gearbox vibration (when the vibration of the gearbox is far from the expected)
 
All these three anomalies (or violations) depend on many variables while the turbine is working. Thus, in order to address that, let's use a ML model called [Autoencoder](https://en.wikipedia.org/wiki/Autoencoder), with correlated features. This model is unsupervised. It learns the latent representation of the dataset and tries to predict (regression) the same tensor given as input. The strategy then is to use a dataset collected from a normal turbine (without anomalies). The model will then learn **'what is a normal turbine'**. When the sensors readings of a malfunctioning turbine is used as input, the model will not be able to rebuild the input, predicting something with a high error and detected as an anomaly.

The sequence of the sensors readings can be seen as a time-series dataset and therefore we observe a high correlation between neighbour samples. We can explore this by reformatting the data as a multidimensional tensor. We'll create a temporal encoding of six features in 10x10 steps of 250ms each. 250ms is the interval computed using 5 samples (the time interval between each sample is ~50ms). It means that we will create a tensor with a shape of 6x10x10.

![Tensor](../imgs/tensor.png)

In the tensor above, each color is a different feature, encoded in 100 (10x10) timesteps (from the current reading to the past in a sliding window).

Let's start preparing our dataset, then.

### Install this lib to improve data visualization

In [ ]:
!pip install -U matplotlib seaborn==0.11.1 pywavelets==1.1.1

### Download the raw data

In [ ]:
!mkdir -p data
!curl https://aws-ml-blog.s3.amazonaws.com/artifacts/monitor-manage-anomaly-detection-model-wind-turbine-fleet-sagemaker-neo/dataset_wind_turbine.csv.gz -o data/dataset_wind.csv.gz

## Preparing the dataset

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import pandas as pd
import math
import matplotlib.pyplot as plt
import pywt
import numpy as np
from datetime import datetime

### Helper functions

In [ ]:
def euler_from_quaternion(x, y, z, w):
    """
    Convert a quaternion into euler angles (roll, pitch, yaw)
    roll is rotation around x in radians (counterclockwise)
    pitch is rotation around y in radians (counterclockwise)
    yaw is rotation around z in radians (counterclockwise)
    """
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    roll_x = math.atan2(t0, t1)

    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    pitch_y = math.asin(t2)

    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    yaw_z = math.atan2(t3, t4)

    return roll_x, pitch_y, yaw_z # in radians

In [ ]:
def wavelet_denoise(data, wavelet, noise_sigma):
    '''Filter accelerometer data using wavelet denoising

    Modification of F. Blanco-Silva's code at: https://goo.gl/gOQwy5
    '''
    
    wavelet = pywt.Wavelet(wavelet)
    levels  = min(5, (np.floor(np.log2(data.shape[0]))).astype(int))
    
    # Francisco's code used wavedec2 for image data
    wavelet_coeffs = pywt.wavedec(data, wavelet, level=levels)
    threshold = noise_sigma*np.sqrt(2*np.log2(data.size))

    new_wavelet_coeffs = map(lambda x: pywt.threshold(x, threshold, mode='soft'), wavelet_coeffs)

    return pywt.waverec(list(new_wavelet_coeffs), wavelet)

### Preparing the dataset

In [ ]:
parser = lambda date: datetime.strptime(date, '%Y-%m-%dT%H:%M:%S.%f+00:00')
df = pd.read_csv('data/dataset_wind.csv.gz', compression="gzip", sep=',', low_memory=False, parse_dates=[ 'eventTime'], date_parser=parser)

df.head()

Features:
  - **nanoId**: id of the edge device that collected the data
  - **turbineId**: id of the turbine that produced this data
  - **arduino_timestamp**: timestamp of the arduino that was operating this turbine
  - **nanoFreemem**: amount of free memory in bytes
  - **eventTime**: timestamp of the row
  - **rps**: rotation of the rotor in Rotations Per Second
  - **voltage**: voltage produced by the generator in milivolts
  - **qw, qx, qy, qz**: quaternion angular acceleration
  - **gx, gy, gz**: gravity acceleration
  - **ax, ay, az**: linear acceleration
  - **gearboxtemp**: internal temperature
  - **ambtemp**: external temperature
  - **humidity**: air humidity
  - **pressure**: air pressure
  - **gas**: air quality
  - **wind_speed_rps**: wind speed in Rotations Per Second

In [ ]:
print('now converting quat to euler...')
roll,pitch,yaw = [], [], []
for idx, row in df.iterrows():
    r,p,y = euler_from_quaternion(row['qx'], row['qy'], row['qz'], row['qw'])
    roll.append(r)
    pitch.append(p)
    yaw.append(y)
df['roll'] = roll
df['pitch'] = pitch
df['yaw'] = yaw

In [ ]:
## we will select the following features to prepare our dataset
## with these features we have parameters for vibration, rotation and voltage
quat=['qx', 'qy', 'qz', 'qw']
rot=['wind_speed_rps', 'rps']
volt=['voltage']
features = quat + rot + volt

### Ploting the vibration data, just to have an idea

In [ ]:
df[quat[:3]].iloc[1910:2000].plot(figsize=(20,10))

### Now, plot the rotation of the turbine and the wind speed in RPS

In [ ]:
df[rot].iloc[1910:2000].plot(figsize=(20,10))

### Finally, plot the voltage readings

In [ ]:
df[volt].iloc[1910:2000].plot(figsize=(20,10))

### Cleaning and normalizing

In [ ]:
df_train = df.copy()

# select the features
features = ['roll', 'pitch', 'yaw', 'wind_speed_rps', 'rps', 'voltage']

# get the std for denoising
raw_std = df_train[features].std()
for f in features:
    df_train[f] = wavelet_denoise(df_train[f].values, 'db6', raw_std[f])#[:-1]

# normalize
training_std = df_train[features].std()
training_mean = df_train[features].mean()
df_train = (df_train[features] - training_mean) / training_std

print("raw_std = np.array([")
for k in raw_std.keys(): print("    %f, # %s" % (raw_std[k], k))
print("])")

print("mean = np.array([")
for k in training_mean.keys(): print("    %f, # %s" % (training_mean[k], k))
print("])")
print("std = np.array([")
for k in training_std.keys(): print("    %f, # %s" % (training_std[k], k))
print("])")

print("Number of training samples:", len(df_train))
df_train.head()

### Alright, this is our dataset. Let's just plot the original vs the prepared data
**Original Data**

In [ ]:
df[features][:2000].plot(figsize=(20,10))

**Denoised & Normalized Data**

In [ ]:
df_train[:2000].plot(figsize=(20,10))

In [ ]:
import seaborn as sns
corr = df_train.corr()

f, ax = plt.subplots(figsize=(15, 8))
sns.heatmap(corr, annot=True, fmt="f",
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values,
            ax=ax)

The correlation between rps and voltage is high, but remember that we need both to detect anomalies

In [ ]:
def create_dataset(X, time_steps=1, step=1):
    Xs = []
    for i in range(0, len(X) - time_steps, step):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
    return np.array(Xs)

In [ ]:
INTERVAL = 5 # seconds
TIME_STEPS = 20 * INTERVAL # 50ms -> seg: 50ms * 20
STEP = 10
n_cols = len(df_train.columns)
X = create_dataset(df_train, TIME_STEPS, STEP)
X = np.nan_to_num(X, copy=True, nan=0.0, posinf=None, neginf=None)

X = np.transpose(X, (0, 2, 1)).reshape(X.shape[0], n_cols, 10, 10)

X.shape

In [ ]:
## We need to split the array in chunks of at most 5MB
!rm -rf data/*.npy
for i,x in enumerate(np.array_split(X, 60)):
    np.save('data/wind_turbine_%02d.npy' % i, x)